# Convolutional Neural Network

In [0]:
import tensorflow as tf
import numpy as np

Import mnist data

In [0]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [0]:
# parse label data

def one_hot(oringinal_label):
  datalength = len(oringinal_label)
  label = np.zeros([datalength,10])
  for i in range(len(oringinal_label)):
    index = oringinal_label[i] 
    label[i,index]=1    
  return label


train_labels = one_hot(train_labels)
eval_labels = one_hot(eval_labels)

In [0]:
train_labels.shape

(55000, 10)

In [0]:
# hidden layer
def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    Weights = tf.Variable(tf.random_normal([in_size, out_size],stddev=0.01))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    Wx_plus_b = tf.nn.dropout(Wx_plus_b, rate=0.5)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs

In [0]:
# convolutional and pooling layer
def conv_layer(inputs, inputs_shape, nfilters):
  new_inputs = tf.reshape(inputs, inputs_shape)
  layer_output = tf.layers.conv2d(new_inputs, filters=nfilters, kernel_size=[5,5], padding='same')
  return layer_output


def maxpool(inputs):
  layer_output = tf.nn.max_pool(inputs, [1,2,2,1], [1,2,2,1], 'VALID')
  return layer_output

Set the placeholder.

In [0]:
InputNeurons = 784
OutputNeurons = 10
drop_rate = 0.5  # dropout parameter, keep how many percentage
learning_rate = 0.01
printstep = 50
traing_epochs = 701

# define placeholder for inputs to network
rate = tf.placeholder(tf.float32)
xs = tf.placeholder(tf.float32, [None, InputNeurons])
ys = tf.placeholder(tf.float32, [None, OutputNeurons])

Define the structure of the convolutional neural network.

In [0]:
# add hidden layer
conv1 = conv_layer(xs, [-1,28,28,1], 9)
mp1 = maxpool(conv1)
conv2 = conv_layer(mp1, [-1,14,14,1], 9)
mp2 = maxpool(conv2)
mp2 = tf.reshape(mp2, [-1,7*7*9*9])
l1 = add_layer(mp2, 7*7*9*9, 100, activation_function=tf.nn.relu) 
prediction = add_layer(l1, 100, OutputNeurons, activation_function=tf.nn.softmax)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# the error between prediction and real data
#loss = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), axis=1))
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=ys,logits=prediction,axis=-1)
train_step = tf.train.AdagradOptimizer(learning_rate).minimize(loss)

# start of the neural network
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(traing_epochs):
    # training
    idx = np.random.choice(np.arange(len(train_data)), 64, replace=False)
    x_train_batch = train_data[idx]
    y_train_batch = train_labels[idx]
    sess.run(train_step, feed_dict={xs: x_train_batch, ys: y_train_batch, rate: drop_rate})
    if i % printstep == 0:
        # to see the step improvement
        train_loss = sess.run(loss, feed_dict={xs: x_train_batch, ys: y_train_batch, rate: 1})
        validation_loss = sess.run(loss, feed_dict={xs: eval_data, ys: eval_labels, rate:1})
        print("epoch",i,"validation loss=", validation_loss,"Train Loss=",train_loss)

epoch 0 validation loss= [2.2960289 2.2877831 2.3051462 ... 2.2949636 2.3135176 2.3066769] Train Loss= [2.292575  2.3136723 2.2901454 2.290981  2.3072474 2.3097942 2.2883775
 2.3136477 2.311834  2.3136353 2.288071  2.3090732 2.2972393 2.2953444
 2.2858863 2.3107464 2.2930558 2.2840364 2.3043816 2.3152888 2.3134217
 2.3116295 2.3090253 2.308257  2.3143618 2.3104026 2.3106887 2.3097122
 2.294181  2.3128924 2.2882855 2.3129556 2.29972   2.3157697 2.2861457
 2.2923927 2.2957098 2.3096204 2.286984  2.3127615 2.299711  2.2928395
 2.2897594 2.3093054 2.2871878 2.3063679 2.3163524 2.2886393 2.3122463
 2.313577  2.2898605 2.2927845 2.2795823 2.3141525 2.2916133 2.296153
 2.3167343 2.281901  2.3150797 2.3078153 2.2942948 2.2843156 2.287541
 2.3151588]
epoch 50 validation loss= [2.3855221 2.3731287 1.4803826 ... 2.3620608 2.377378  2.2892919] Train Loss= [2.351445  2.3036413 2.4142869 2.3920321 2.4469805 2.3022225 2.3788671
 2.327118  2.3292158 2.159106  2.271593  2.2755232 2.005219  2.3779917
 2